In [1]:
import pandas as pd
from typing import List, Optional

def load_pickle(pickle_path: str) -> pd.DataFrame:
    """
    Load a DataFrame from a pickle file.

    :param pickle_path: Path to the pickle file.
    :return: Loaded pandas DataFrame.
    """
    df = pd.read_pickle(pickle_path)
    print(f"Loaded data from {pickle_path} with shape {df.shape}")
    return df

def basic_info(df: pd.DataFrame):
    """
    Display basic information about the DataFrame.

    :param df: pandas DataFrame.
    """
    print("\n--- Basic Info ---")
    print(df.info())
    print("\n--- First 5 rows ---")
    print(df.head())
    print("\n--- Data Description ---")
    print(df.describe(include='all'))

def missing_values_summary(df: pd.DataFrame):
    """
    Print a summary of missing values per column.

    :param df: pandas DataFrame.
    """
    print("\n--- Missing Values ---")
    missing = df.isna().sum()
    missing_percent = (missing / len(df)) * 100
    summary = pd.DataFrame({"missing_count": missing, "missing_percent": missing_percent})
    print(summary[summary["missing_count"] > 0])

def duplicates_summary(df: pd.DataFrame):
    """
    Print the number of duplicate rows.

    :param df: pandas DataFrame.
    """
    dup_count = df.duplicated().sum()
    print(f"\n--- Duplicates ---\nNumber of duplicate rows: {dup_count}")

def unique_values_summary(df: pd.DataFrame, columns: Optional[List[str]] = None):
    """
    Print the number of unique values per column or for selected columns.

    :param df: pandas DataFrame.
    :param columns: List of columns to check unique values for. If None, checks all columns.
    """
    columns_to_check = columns if columns else df.columns
    print("\n--- Unique Values ---")
    for col in columns_to_check:
        print(f"{col}: {df[col].nunique()} unique values")

def value_counts_summary(df: pd.DataFrame, columns: Optional[List[str]] = None, top_n: int = 10):
    """
    Print the top N frequent values for categorical columns.

    :param df: pandas DataFrame.
    :param columns: List of columns to analyze. If None, checks all object or category columns.
    :param top_n: Number of top frequent values to show.
    """
    if columns is None:
        columns = df.select_dtypes(include=['object', 'category']).columns

    print("\n--- Top Value Counts ---")
    for col in columns:
        print(f"\nColumn: {col}")
        print(df[col].value_counts().head(top_n))

def correlation_summary(df: pd.DataFrame):
    """
    Print correlation matrix for numeric columns only.
    """
    print("\n--- Correlation Matrix (Numeric Columns Only) ---")
    numeric_df = df.select_dtypes(include=['number'])
    if numeric_df.empty:
        print("No numeric columns to compute correlation.")
    else:
        print(numeric_df.corr())

In [2]:
# Load pickle
df_bluebikes = load_pickle("../data_pipeline/data/processed/bluebikes/station_id_mapping.pkl")


Loaded data from ../data_pipeline/data/processed/bluebikes/station_id_mapping.pkl with shape (3568112, 13)


In [3]:
basic_info(df_bluebikes)


--- Basic Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3568112 entries, 0 to 3568111
Data columns (total 13 columns):
 #   Column                   Dtype              
---  ------                   -----              
 0   ride_id                  object             
 1   rideable_type            object             
 2   start_time               datetime64[ns, UTC]
 3   stop_time                datetime64[ns, UTC]
 4   start_station_name       string             
 5   start_station_id         float64            
 6   end_station_name         string             
 7   end_station_id           float64            
 8   start_station_latitude   float64            
 9   start_station_longitude  float64            
 10  end_station_latitude     float64            
 11  end_station_longitude    float64            
 12  user_type                string             
dtypes: datetime64[ns, UTC](2), float64(6), object(2), string(3)
memory usage: 353.9+ MB
None

--- First 5 rows ---
 

In [3]:
missing_values_summary(df_bluebikes)


--- Missing Values ---
                  missing_count  missing_percent
start_station_id           1237         0.034690
end_station_id             3915         0.109792
ëÀ¼ï                    3565820       100.000000


In [4]:
duplicates_summary(df_bluebikes)


--- Duplicates ---
Number of duplicate rows: 0


In [1]:
# Run this in your Airflow environment or Jupyter notebook
import pandas as pd
import numpy as np

# Load your processed data
bluebike_data = pd.read_pickle('../data_pipeline/data/processed/bluebikes/after_duplicates.pkl')

# Prepare the data
bluebike_data['start_time'] = pd.to_datetime(bluebike_data['start_time'])
bluebike_data['hour'] = bluebike_data['start_time'].dt.hour
bluebike_data['day_of_week'] = bluebike_data['start_time'].dt.dayofweek
bluebike_data['date'] = bluebike_data['start_time'].dt.date

# Calculate ACTUAL hourly rides by hour and day type
hourly_stats = bluebike_data.groupby(['date', 'hour']).size().reset_index(name='ride_count')
hourly_stats['date'] = pd.to_datetime(hourly_stats['date'])
hourly_stats['day_of_week'] = hourly_stats['date'].dt.dayofweek
hourly_stats['is_weekend'] = hourly_stats['day_of_week'].isin([5, 6])

# WEEKDAY STATISTICS
weekday_by_hour = hourly_stats[~hourly_stats['is_weekend']].groupby('hour')['ride_count'].agg(['mean', 'median', 'std'])
print("\n📊 WEEKDAY HOURLY RIDES (System-Wide):")
print(weekday_by_hour)

# WEEKEND STATISTICS
weekend_by_hour = hourly_stats[hourly_stats['is_weekend']].groupby('hour')['ride_count'].agg(['mean', 'median', 'std'])
print("\n📊 WEEKEND HOURLY RIDES (System-Wide):")
print(weekend_by_hour)

# Save as JSON for easy use in JavaScript
import json

weekday_lookup = weekday_by_hour['mean'].round().to_dict()
weekend_lookup = weekend_by_hour['mean'].round().to_dict()

historical_patterns = {
    'weekday': {int(k): int(v) for k, v in weekday_lookup.items()},
    'weekend': {int(k): int(v) for k, v in weekend_lookup.items()}
}

with open('historical_patterns.json', 'w') as f:
    json.dump(historical_patterns, f, indent=2)

print("\n✅ Saved to historical_patterns.json")
print(json.dumps(historical_patterns, indent=2))


📊 WEEKDAY HOURLY RIDES (System-Wide):
             mean  median         std
hour                                 
0.0     89.798851    83.0   54.323928
1.0     51.261494    48.0   35.082907
2.0     26.393678    24.0   17.537203
3.0     14.332370    13.0    8.761705
4.0     20.620690    20.0    9.937837
5.0     86.497126    82.0   41.133192
6.0    254.479885   242.0  115.175431
7.0    616.284483   596.5  282.592521
8.0   1087.862069  1081.5  459.590431
9.0    699.192529   720.0  268.093795
10.0   479.258621   492.5  198.233704
11.0   501.425287   520.5  212.355685
12.0   581.979885   602.0  259.676605
13.0   593.017241   586.5  268.504772
14.0   636.186782   651.5  279.616817
15.0   784.445402   786.5  330.099286
16.0  1101.120690  1104.5  451.302364
17.0  1458.879310  1430.5  639.879777
18.0  1140.540230  1121.0  533.860311
19.0   824.091954   794.0  403.040115
20.0   574.951149   526.5  297.039223
21.0   433.954023   416.5  231.981660
22.0   322.848138   282.0  203.715960
23.0   202.

In [2]:
print(f"Total rides: {len(bluebike_data):,}")

# Calculate rides per station
station_rides = bluebike_data.groupby('start_station_id').size().reset_index(name='ride_count')
total_rides = station_rides['ride_count'].sum()

# Calculate share percentage
station_rides['share'] = station_rides['ride_count'] / total_rides
station_rides = station_rides.sort_values('share', ascending=False)

print("\n📊 TOP 20 BUSIEST STATIONS:")
print(station_rides.head(20)[['start_station_id', 'ride_count', 'share']])

print("\n📊 STATION SHARE STATISTICS:")
print(station_rides['share'].describe())

# Convert to lookup dictionary
station_shares = dict(zip(
    station_rides['start_station_id'].astype(str),
    station_rides['share'].round(6)
))

# Save as JSON
with open('station_shares.json', 'w') as f:
    json.dump(station_shares, f, indent=2)

print(f"\n✅ Saved {len(station_shares)} station shares to station_shares.json")

# Show some examples
print("\nExample station shares:")
for station_id, share in list(station_shares.items())[:10]:
    print(f"  {station_id}: {share:.4f} ({share*100:.2f}%)")

Total rides: 8,182,157

📊 TOP 20 BUSIEST STATIONS:
     start_station_id  ride_count     share
191             192.0      143965  0.017595
36               37.0      113122  0.013825
12               13.0       99246  0.012130
31               32.0       89078  0.010887
193             194.0       79934  0.009769
76               77.0       78506  0.009595
0                 1.0       71240  0.008707
327             328.0       69441  0.008487
320             321.0       66950  0.008182
363             364.0       66896  0.008176
435             436.0       65746  0.008035
219             220.0       64474  0.007880
50               51.0       61257  0.007487
34               35.0       59625  0.007287
274             275.0       58395  0.007137
78               79.0       57044  0.006972
19               20.0       56209  0.006870
207             208.0       52900  0.006465
116             117.0       52364  0.006400
532             533.0       52343  0.006397

📊 STATION SHARE STATISTI

In [3]:
print("Column names:")
print(bluebike_data.columns.tolist())

print("\nFirst 5 rows of station-related columns:")
print(bluebike_data[['start_station_id', 'start_station_name']].head(10))

print("\nUnique start_station_names (first 20):")
print(bluebike_data['start_station_name'].unique()[:20])

print("\nData types:")
print(bluebike_data[['start_station_id', 'start_station_name']].dtypes)

Column names:
['ride_id', 'rideable_type', 'start_time', 'stop_time', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude', 'user_type', 'hour', 'day_of_week', 'date']

First 5 rows of station-related columns:
   start_station_id              start_station_name
0               1.0              Ames St at Main St
1               1.0              Ames St at Main St
2               2.0              One Memorial Drive
3               1.0              Ames St at Main St
4               3.0              Mass Ave T Station
5               4.0  St. Alphonsus St at Tremont St
6               1.0              Ames St at Main St
7               1.0              Ames St at Main St
8               1.0              Ames St at Main St
9               1.0              Ames St at Main St

Unique start_station_names (first 20):
<StringArray>
[                               'Ame

In [4]:

print(f"Total rides: {len(bluebike_data):,}")

# Calculate rides per station NAME
station_rides = bluebike_data.groupby('start_station_name').size().reset_index(name='ride_count')
total_rides = station_rides['ride_count'].sum()

# Calculate share percentage
station_rides['share'] = station_rides['ride_count'] / total_rides
station_rides = station_rides.sort_values('share', ascending=False)

print("\n📊 TOP 20 BUSIEST STATIONS (BY NAME):")
print(station_rides.head(20))

# Convert to lookup dictionary BY NAME
station_shares_by_name = dict(zip(
    station_rides['start_station_name'].astype(str),
    station_rides['share'].round(6)
))

# Save as JSON
with open('station_shares_by_name.json', 'w') as f:
    json.dump(station_shares_by_name, f, indent=2)

print(f"\n✅ Saved {len(station_shares_by_name)} station shares to station_shares_by_name.json")

# Show some examples
print("\nExample station shares BY NAME:")
for name, share in list(station_shares_by_name.items())[:10]:
    print(f"  {name}: {share:.4f} ({share*100:.2f}%)")


Total rides: 8,182,157

📊 TOP 20 BUSIEST STATIONS (BY NAME):
                                    start_station_name  ride_count     share
368                       MIT at Mass Ave / Amherst St      143965  0.017595
142              Central Square at Mass Ave / Essex St      113122  0.013825
297                Harvard Square at Mass Ave/ Dunster       99246  0.012130
367                                      MIT Vassar St       89078  0.010887
365                    MIT Pacific St at Purrington St       79934  0.009769
150        Charles Circle - Charles St at Cambridge St       78506  0.009595
43                                  Ames St at Main St       71240  0.008707
165  Christian Science Plaza - Massachusetts Ave at...       69441  0.008487
97                         Boylston St at Fairfield St       66950  0.008182
391                          Mass Ave/Lafayette Square       66896  0.008176
67                      Beacon St at Massachusetts Ave       65746  0.008035
525            

In [9]:
value_counts_summary(df_bluebikes)


--- Top Value Counts ---

Column: ride_id
ride_id
D39FCF4FF35223A0    1
215E35C14D594F94    1
FFCD2D3BA2776BEE    1
48EE2AAAB3450EED    1
CEBF456BE80209BF    1
F42C7FC3627D16D3    1
57C96B6DB6A2DA59    1
1B6BA97C1D513C60    1
6E4A19B4F9EA5957    1
3B1114528642D4F3    1
Name: count, dtype: int64

Column: rideable_type
rideable_type
classic_bike     2570910
electric_bike     994910
Name: count, dtype: int64

Column: ëÀ¼ï
Series([], Name: count, dtype: int64)


In [10]:
correlation_summary(df_bluebikes)


--- Correlation Matrix (Numeric Columns Only) ---
                         start_station_latitude  start_station_longitude  \
start_station_latitude                 1.000000                -0.001213   
start_station_longitude               -0.001213                 1.000000   
end_station_latitude                   0.749335                 0.019133   
end_station_longitude                  0.018261                 0.717820   

                         end_station_latitude  end_station_longitude  
start_station_latitude               0.749335               0.018261  
start_station_longitude              0.019133               0.717820  
end_station_latitude                 1.000000              -0.005286  
end_station_longitude               -0.005286               1.000000  


In [11]:
# Load pickle
df = load_pickle("../data_pipeline/data/processed/boston_clg/raw_data.pkl")


Loaded data from ../data_pipeline/data/processed/boston_clg/raw_data.pkl with shape (60, 28)


In [13]:
basic_info(df)


--- Basic Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   OBJECTID       60 non-null     int64  
 1   Match_type     60 non-null     object 
 2   Ref_ID         60 non-null     int64  
 3   ID1            44 non-null     object 
 4   Id             60 non-null     object 
 5   SchoolId       60 non-null     object 
 6   Name           60 non-null     object 
 7   Address        60 non-null     object 
 8   City           60 non-null     object 
 9   Zipcode        58 non-null     float64
 10  Contact        60 non-null     object 
 11  PhoneNumbe     60 non-null     object 
 12  YearBuilt      60 non-null     object 
 13  NumStories     60 non-null     object 
 14  Cost           60 non-null     float64
 15  NumStudent     60 non-null     float64
 16  BackupPowe     60 non-null     float64
 17  ShelterCap     60 non-null     objec

In [12]:
missing_values_summary(df)


--- Missing Values ---
              missing_count  missing_percent
ID1                      16        26.666667
Zipcode                   2         3.333333
X                         3         5.000000
Y                         3         5.000000
NumStudent12             22        36.666667
CampusHous               13        21.666667
URL                      12        20.000000
Address2013              60       100.000000


In [14]:
duplicates_summary(df)


--- Duplicates ---
Number of duplicate rows: 0


In [15]:
unique_values_summary(df)


--- Unique Values ---
OBJECTID: 60 unique values
Match_type: 4 unique values
Ref_ID: 1 unique values
ID1: 39 unique values
Id: 60 unique values
SchoolId: 44 unique values
Name: 59 unique values
Address: 60 unique values
City: 19 unique values
Zipcode: 53 unique values
Contact: 47 unique values
PhoneNumbe: 56 unique values
YearBuilt: 22 unique values
NumStories: 15 unique values
Cost: 56 unique values
NumStudent: 42 unique values
BackupPowe: 1 unique values
ShelterCap: 1 unique values
Latitude: 58 unique values
Longitude: 58 unique values
Comment: 58 unique values
X: 57 unique values
Y: 56 unique values
NumStudent12: 38 unique values
CampusHous: 17 unique values
NumStudents13: 43 unique values
URL: 46 unique values
Address2013: 0 unique values


In [16]:
value_counts_summary(df)


--- Top Value Counts ---

Column: Match_type
Match_type
NCES & Consortium    28
Consortium           16
NCES                 13
                      3
Name: count, dtype: int64

Column: ID1
ID1
                                                      5
300 The Fenway, Boston, Massachusetts 02115-5898      2
Fruit St, Boston, Massachusetts 02114-2696            1
250 New Rutherford Ave, Boston, Massachusetts 021*    1
36 1st Avenue, Boston, Massachusetts 02129-4557       1
8 Ashburton Place, Boston, Massachusetts 02108-27*    1
41 Berkeley St, Boston, Massachusetts 02116           1
230 The Fenway, Boston, Massachusetts 02115-5596      1
400 The Fenway, Boston, Massachusetts 02115           1
8 the Fenway, Boston, Massachusetts 02215             1
Name: count, dtype: int64

Column: Id
Id
98627    1
11959    1
16632    1
1949     1
9109     1
86640    1
86302    1
86125    1
15422    1
22285    1
Name: count, dtype: int64

Column: SchoolId
SchoolId
0         15
164845     2
167543     2
4

In [17]:
correlation_summary(df)


--- Correlation Matrix (Numeric Columns Only) ---
               OBJECTID  Ref_ID   Zipcode      Cost  NumStudent  BackupPowe  \
OBJECTID       1.000000     NaN -0.171385 -0.076258   -0.107168         NaN   
Ref_ID              NaN     NaN       NaN       NaN         NaN         NaN   
Zipcode       -0.171385     NaN  1.000000  0.190170    0.458679         NaN   
Cost          -0.076258     NaN  0.190170  1.000000    0.405653         NaN   
NumStudent    -0.107168     NaN  0.458679  0.405653    1.000000         NaN   
BackupPowe          NaN     NaN       NaN       NaN         NaN         NaN   
Latitude      -0.620160     NaN  0.188255  0.093444    0.098388         NaN   
Longitude      0.619804     NaN -0.188061 -0.093593   -0.098415         NaN   
X             -0.256362     NaN  0.103841  0.069055    0.011929         NaN   
Y             -0.361367     NaN  0.016144 -0.120060   -0.021006         NaN   
NumStudent12  -0.274917     NaN  0.600122  0.720564    0.820517         NaN   
N

In [18]:
# Load pickle
df = load_pickle("../data_pipeline/data/processed/NOAA_weather/raw_data.pkl")


Loaded data from ../data_pipeline/data/processed/NOAA_weather/raw_data.pkl with shape (3946, 4)


In [19]:
basic_info(df) 


--- Basic Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3946 entries, 0 to 3945
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    3946 non-null   object 
 1   PRCP    3946 non-null   float64
 2   TMAX    3946 non-null   float64
 3   TMIN    3946 non-null   float64
dtypes: float64(3), object(1)
memory usage: 123.4+ KB
None

--- First 5 rows ---
         date  PRCP  TMAX  TMIN
0  2015-01-01   0.0   0.6  -5.5
1  2015-01-02   0.0   5.0  -0.5
2  2015-01-03  15.7   2.8  -5.5
3  2015-01-04  14.5  11.1   1.7
4  2015-01-05   0.0  10.0  -8.2

--- Data Description ---
              date         PRCP         TMAX         TMIN
count         3946  3946.000000  3946.000000  3946.000000
unique        3946          NaN          NaN          NaN
top     2025-10-20          NaN          NaN          NaN
freq             1          NaN          NaN          NaN
mean           NaN     2.983730    16.118550     7.839128
std  

In [20]:
missing_values_summary(df)


--- Missing Values ---
Empty DataFrame
Columns: [missing_count, missing_percent]
Index: []


In [21]:
duplicates_summary(df)


--- Duplicates ---
Number of duplicate rows: 0


In [22]:
value_counts_summary(df)


--- Top Value Counts ---

Column: date
date
2025-10-20    1
2015-01-01    1
2015-01-02    1
2015-01-03    1
2015-01-04    1
2015-01-05    1
2015-01-06    1
2015-01-07    1
2015-01-08    1
2015-01-09    1
Name: count, dtype: int64


In [23]:
correlation_summary(df)


--- Correlation Matrix (Numeric Columns Only) ---
          PRCP      TMAX      TMIN
PRCP  1.000000 -0.026179  0.017909
TMAX -0.026179  1.000000  0.944070
TMIN  0.017909  0.944070  1.000000
